In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

# Creating Database


In [2]:
con = sqlite3.connect("f1.db")
cur = con.cursor()
for path in os.listdir("./data"):
    data = pd.read_csv("./data/" + path)
    data.to_sql(path[:-4], con, if_exists="replace", index=False)

In [3]:
cur.execute("CREATE TABLE standings_plus AS select driverStandingsId, driver_standings.raceId, driverId, points, position, positionText, wins, year, round, circuitId from driver_standings INNER JOIN races on driver_standings.raceId=races.raceId;").fetchall()

[]

In [4]:
cur.execute("SELECT MIN(year) from (SELECT year, MAX(position) as pos FROM standings_plus GROUP BY year) WHERE pos < 30;").fetchall()

[(1996,)]

In [5]:
def reLU(arr):
    return np.maximum(0,arr)

def sigmoid(arr):
    return 1 / (1 + np.exp(-arr))

def cost(guess, answer):
    return np.sum(np.square(guess - answer))

class Network:
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.B = [np.random.randn(y) for y in sizes[1:]]
        self.W = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def evaluate(self, v_in, squish=sigmoid):
        self.A = [np.zeros(x) for x in self.sizes]
        self.A[0] = v_in
        for i in range(1, self.num_layers):
            self.A[i] = squish(self.W[i - 1]@self.A[i - 1] + self.B[i - 1])
        return self.A[-1]

In [6]:
data = pd.read_sql("SELECT * FROM results JOIN races ON results.raceId=races.raceId WHERE year >=1996;",con)
x = data[['driverId','constructorId','grid','circuitId']]
y = data['positionOrder']

In [7]:
Y = np.zeros((len(y),24))
for i,n in y.items():
    Y[i,n-1] = 1
Y[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [8]:
N = Network([4,16,16,24])

In [12]:
cost(N.evaluate(x.values[0]), Y[0])

8.611033861602163

In [100]:
Y[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [86]:
N.A[0]

array([18, 23,  1,  1])